In [ ]:
import requests
import json
import time
from datetime import datetime


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # e.g., 20250612_023015

with open("yelp_api_key.txt", "r") as f:
    yelp_api_key = f.read().strip()
    
url = 'https://api.yelp.com/v3/businesses/search'


headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {yelp_api_key}"
}

final_raw_data=[]

for i in range(0, 1000, 50):
    params = {
        "location": "Orange County",
        "term": "asian food",
        "sort_by": "best_match",
        "limit": 50,
        "offset": i
    }

    response = requests.get(url, headers=headers, params=params)
    raw_data = response.json()
    final_raw_data.extend(raw_data.get("businesses",[]))
    time.sleep(0.25)


filename = f"yelp_orange_county_asian_food_{timestamp}.json"

with open(filename, "w") as f:
    for i in final_raw_data:
        json.dump(i, f)
        f.write("\n")


print(f"✅ Data saved to {filename}")
print("Total: ",len(final_raw_data))
#final_raw_data


✅ Data saved to yelp_orange_county_asian_food_20250613_080229.json
Total:  200


In [51]:
print (filename)

yelp_orange_county_asian_food_20250613_080229.json


In [58]:
with open("yelp_api_key.txt", "r") as f:
    yelp_api_key = f.read().strip()

In [52]:
import os
print(os.getcwd())

/home/devuser/project/Scripts


In [53]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f"/home/devuser/project/{filename}"
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

/home/devuser/project/yelp_orange_county_asian_food_20250613_080229.json


In [54]:
# load the json file into google cloud storage

from google.cloud import storage
import os

# Set path to service account key file (optional if already exported)
credential_filename = "yelpdaily-58438c6c49cc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f"/home/devuser/project/{credential_filename}"

# Define GCS info
bucket_name = "yelpdailydata"  # <-- replace with your GCS bucket name
source_file_path = f"/home/devuser/project/scripts/{filename}"  # local path
destination_blob_name = f"yelp/{filename}"  # GCS path

# Initialize GCS client
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

# Upload the file
blob.upload_from_filename(source_file_path)

print(f"✅ File uploaded to gs://{bucket_name}/{destination_blob_name}")


✅ File uploaded to gs://yelpdailydata/yelp/yelp_orange_county_asian_food_20250613_080229.json


In [55]:
from google.cloud import bigquery
import os

# Set path to service account key file (optional if already exported)
credential_filename = "yelpdaily-58438c6c49cc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f"/home/devuser/project/{credential_filename}"

# Initialize the BigQuery client
client = bigquery.Client()

# Set your variables
bucket_name = "yelpdailydata"  # Replace with your GCS bucket name
file_name = f"{filename}"
gcs_path = f"gs://{bucket_name}/yelp/{file_name}"  # Path to your JSON file
dataset_id = "yelpdaily"  # Replace with your BigQuery dataset name
table_id = f"{file_name}".removesuffix(".json")  # Replace with your target table name

# Full table ID in project.dataset.table format
project = client.project
table_ref = f"{project}.{dataset_id}.{table_id}"

# Configure the load job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True  # Automatically detect schema from JSON data
)

# Start the load job
load_job = client.load_table_from_uri(
    gcs_path,
    table_ref,
    job_config=job_config
)

print("Starting job...")
load_job.result()  # Wait for the job to finish

print(f"Loaded {load_job.output_rows} rows into {table_ref}")


Starting job...
Loaded 200 rows into yelpdaily.yelpdaily.yelp_orange_county_asian_food_20250613_080229
